In [13]:
import os
from clickhouse_driver import Client
import logging
from datetime import timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
flight_hours_table_name = 'flight_hours'
olap_table_name = 'OlapCube_VNV'

# Функция для заполнения поля daily_flight_hours в таблице OLAP
def update_daily_flight_hours(client):
    try:
        # Получение значения daily_flight_hours из таблицы flight_hours
        flight_hours_query = f"""
            SELECT daily_flight_hours
            FROM {database_name}.{flight_hours_table_name}
            LIMIT 1
        """
        daily_flight_hours = client.execute(flight_hours_query)[0][0]
        logging.info(f"Получено значение daily_flight_hours: {daily_flight_hours}")
    except Exception as e:
        logging.error(f"Ошибка при получении daily_flight_hours из flight_hours: {e}")
        raise

    # Обновление поля daily_flight_hours в OLAP кубе для всех записей на всем протяжении дат
    try:
        update_query = f"""
            ALTER TABLE {database_name}.{olap_table_name}
            UPDATE daily_flight_hours = {daily_flight_hours}
            WHERE daily_flight_hours IS NULL
        """
        client.execute(update_query)
        logging.info(f"Поле daily_flight_hours успешно обновлено в {olap_table_name}")
    except Exception as e:
        logging.error(f"Ошибка при обновлении поля daily_flight_hours в {olap_table_name}: {e}")
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Обновление поля daily_flight_hours в OLAP кубе
try:
    update_daily_flight_hours(client)
    logging.info("Поле daily_flight_hours успешно обновлено в OLAP кубе")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении daily_flight_hours: {e}", exc_info=True)


2024-11-29 11:27:03,139 - INFO - Подключение к ClickHouse успешно установлено
2024-11-29 11:27:03,147 - INFO - Получено значение daily_flight_hours: 1.58
2024-11-29 11:27:03,153 - INFO - Поле daily_flight_hours успешно обновлено в OlapCube_VNV
2024-11-29 11:27:03,154 - INFO - Поле daily_flight_hours успешно обновлено в OLAP кубе
